# Formatação de Dados
- 1. Formatação de Dados
    - 1.1 Leitura de dados
    - 1.2 Verificação os tipos de dados e as informações sobre os dados
    - 1.3 Formatando dados usando astype
- 2. Criando uma função para limpeza de dados
    - 2.1. replace
    - 2.2 ``np.where()``
- 3. Formatando números e datas usando 
    - 3.1 ``pd.to_numeric()``  
    - 3.2 ``pd.to_datetime()``
- 4. Formatando dados no readcsv

<img src='https://pbpython.com/images/pandas_dtypes.png'>

In [2]:
import pandas as pd
import numpy as np

# 1. Formação de Dados

Quando fazemos uma análise e dados, é importante certificar-se de que estamos usando os tipos de dados corretos; caso contrário, você poderá obter resultados ou erros inesperados.

No caso dos pandas, ele inferirá corretamente os tipos de dados em muitos casos e você poderá prosseguir com sua análise sem pensar mais no tópico.

### 1.1 Leitura de Dados

In [5]:
filename = 'data/sales_raw.csv'
df = pd.read_csv(filename)
df.head()

Customer Number     Customer Name         2016          2017  \
0          10002.0  Quest Industries  $125,000.00    $162500.00   
1         552278.0    Smith Plumbing  $920,000.00  $101,2000.00   
2          23477.0   ACME Industrial   $50,000.00     $62500.00   
3          24900.0        Brekke LTD  $350,000.00    $490000.00   
4         651029.0         Harbor Co   $15,000.00     $12750.00   

  Percent Growth Jan Units  Month  Day  Year Active  
0         30.00%       500      1   10  2015      Y  
1         10.00%       700      6   15  2014      Y  
2         25.00%       125      3   29  2016      Y  
3          4.00%        75     10   27  2015      Y  
4        -15.00%    Closed      2    2  2014      N

### 1.2 Verificar tipos de Dados

In [6]:
df.dtypes

Customer Number    float64
Customer Name       object
2016                object
2017                object
Percent Growth      object
Jan Units           object
Month                int64
Day                  int64
Year                 int64
Active              object
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Customer Number  5 non-null      float64
 1   Customer Name    5 non-null      object 
 2   2016             5 non-null      object 
 3   2017             5 non-null      object 
 4   Percent Growth   5 non-null      object 
 5   Jan Units        5 non-null      object 
 6   Month            5 non-null      int64  
 7   Day              5 non-null      int64  
 8   Year             5 non-null      int64  
 9   Active           5 non-null      object 
dtypes: float64(1), int64(3), object(6)
memory usage: 528.0+ bytes


No entanto, em muitos casos, você provavelmente precisará converter dados explicitamente de um tipo para outro.

- O `Customer Number` é um float64, mas deveria ser um int64
- As colunas `2016` e `2017` são armazenadas como `object`, não como valores numéricos (float64 ou int64)
- `Percent Growth` e `Jan Units` também são armazenadas como objetos e não como valores numéricos
- As colunas `Month`, `Day`, and `Year` devem ser convertidas para datetime64
- A coluna `Active` deve ser um booleano

Como converter tipos de dados no pandas?

- 1) Use a função `astype()` para força o tipo de dado apropriado 
- 2) Criar uma função personalizada para conversão de dados 
- 3) Usar as funções como `to_numeric()` or `to_datetime()`

### 1.3 Usando astype

In [8]:
df['Customer Number'].astype('int')

0     10002
1    552278
2     23477
3     24900
4    651029
Name: Customer Number, dtype: int32

Para realmente alterar o número do cliente no dataframe original, certifique-se de atribuí-lo de volta, pois as funções astype() retornam uma cópia.

In [9]:
df["Customer Number"] = df['Customer Number'].astype('int')
df.dtypes

Customer Number     int32
Customer Name      object
2016               object
2017               object
Percent Growth     object
Jan Units          object
Month               int64
Day                 int64
Year                int64
Active             object
dtype: object

In [10]:
# ValueError
df['2016'].astype('float')

ValueError: could not convert string to float: '$125,000.00'

In [11]:
# ValueError
df['Jan Units'].astype('int')

ValueError: invalid literal for int() with base 10: 'Closed'

Ambos retornam exceções ``ValueError`` que significam que as conversões não funcionaram.

Em cada um dos casos, os dados incluíam valores que não podiam ser interpretados como números. Nas colunas de vendas, os dados incluem um símbolo de moeda e uma vírgula em cada valor. Na coluna Jan Unitsm o último valor é “Closed” que não é um número; então obtemos a exceção.

Até agora não parece tão bom para astype() como uma ferramenta. Devemos tentar mais uma vez na coluna Ativo.

In [12]:
df['Active'].astype('bool')

0    True
1    True
2    True
3    True
4    True
Name: Active, dtype: bool

À primeira vista, isso parece bom, mas após uma inspeção mais detalhada, há um grande problema. Todos os valores foram interpretados como True, mas o último cliente tem um sinalizador Ativo de N, portanto, isso não parece correto.

A conclusão desta seção é que ``astype()`` só funcionará se:

- 1) os dados estão limpos e podem ser simplesmente interpretados como um número
- 2) você deseja converter um valor numérico em um objeto de string

Se os dados tiverem caracteres não numéricos ou não forem homogêneos, ``astype()`` não será uma boa opção para conversão de tipo. Você precisará fazer transformações adicionais para que a alteração do tipo de dado funcione corretamente.

# 2. Usando uma função personalizada
- funções personalizadas
- Funções lamdas
- np_where()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer Number  5 non-null      int32 
 1   Customer Name    5 non-null      object
 2   2016             5 non-null      object
 3   2017             5 non-null      object
 4   Percent Growth   5 non-null      object
 5   Jan Units        5 non-null      object
 6   Month            5 non-null      int64 
 7   Day              5 non-null      int64 
 8   Year             5 non-null      int64 
 9   Active           5 non-null      object
dtypes: int32(1), int64(3), object(6)
memory usage: 508.0+ bytes


### 2.1 replace and apply

In [14]:
df['2016']

0    $125,000.00
1    $920,000.00
2     $50,000.00
3    $350,000.00
4     $15,000.00
Name: 2016, dtype: object

In [15]:
df['2016'].str.replace(',','')

0    $125000.00
1    $920000.00
2     $50000.00
3    $350000.00
4     $15000.00
Name: 2016, dtype: object

In [16]:
df['2016'].str.replace(',','', regex=False).str.replace('$','', regex=False)

0    125000.00
1    920000.00
2     50000.00
3    350000.00
4     15000.00
Name: 2016, dtype: object

In [17]:
def convert_currency(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

In [18]:
df['2016'].apply(convert_currency)

0    125000.0
1    920000.0
2     50000.0
3    350000.0
4     15000.0
Name: 2016, dtype: float64

Sucesso! Todos os valores são mostrados como ``float64`` para que possamos fazer todas as funções matemáticas que precisamos.

Tenho certeza de que os leitores mais experientes estão perguntando por que eu não usei apenas uma função ``lambda``? Antes de responder, aqui está o que poderíamos fazer em 1 linha com uma função lambda:

### 2.2 funções lambdas

In [19]:
df['2016'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')

0    125000.0
1    920000.0
2     50000.0
3    350000.0
4     15000.0
Name: 2016, dtype: float64

Usando ``lambda``, podemos simplificar o código em 1 linha, o que é uma abordagem perfeitamente válida. 

Tenho três preocupações principais com essa abordagem:

- 1. As funções lambda são um pouco mais difíceis para o novo usuário entender.
- 2. Se você for usar essa função em várias colunas, prefiro não duplicar a função longa.
- 3. Finalmente, usar uma função facilita a limpeza dos dados ao usar read_csv(). Vou cobrir o uso no final do artigo.

Alguns também podem argumentar que outras abordagens baseadas em lambda apresentam melhorias de desempenho em relação à função personalizada. Isso pode ser verdade, mas para fins de ensinar novos usuários, acho que a abordagem de função é preferível.

Aqui está um exemplo completo de conversão de dados em ambas as colunas de vendas usando a função convert_currency.

In [20]:
df['2016'] = df['2016'].apply(convert_currency)
df['2017'] = df['2017'].apply(convert_currency)

df.dtypes

Customer Number      int32
Customer Name       object
2016               float64
2017               float64
Percent Growth      object
Jan Units           object
Month                int64
Day                  int64
Year                 int64
Active              object
dtype: object

Para outro exemplo de uso de ``lambda`` versus uma função, podemos observar o processo de correção da coluna Percentual de Crescimento.

Usando o ``lambda``:

In [21]:
df['Percent Growth'].apply(lambda x: x.replace('%', '')).astype('float') / 100

0    0.30
1    0.10
2    0.25
3    0.04
4   -0.15
Name: Percent Growth, dtype: float64

Fazendo a mesma coisa usando uma função personalizada

In [22]:
def convert_percent(val):
    """
    Convert the percentage string to an actual floating point percent
    - Remove %
    - Divide by 100 to make decimal
    """
    new_val = val.replace('%', '')
    return float(new_val) / 100

In [23]:
df['Percent Growth'] = df['Percent Growth'].apply(convert_percent)
df['Percent Growth']

0    0.30
1    0.10
2    0.25
3    0.04
4   -0.15
Name: Percent Growth, dtype: float64

A função personalizada final que vamos bordar é ``np.where()`` para converter a coluna ``Active`` em um booleano. Existem várias maneiras possíveis de resolver esse problema específico. A abordagem ``np.where()`` é útil para muitos tipos de problemas, então estou optando por incluí-la aqui.

A ideia básica é usar a função ``np.where()`` para converter todos os valores “Y” para ``True`` e o restante dos valores atribuídos a ``False``

### 2.3 np.where()

In [24]:
np.where(df["Active"] == "Y", True, False)

array([ True,  True,  True,  True, False])

In [25]:
df["Active"] = np.where(df["Active"] == "Y", True, False)

In [26]:
df.dtypes

Customer Number      int32
Customer Name       object
2016               float64
2017               float64
Percent Growth     float64
Jan Units           object
Month                int64
Day                  int64
Year                 int64
Active                bool
dtype: object

# 3. Formatando números e datas usando To_numeric e To_datetime

Pandas tem um meio termo entre a função astype() sem corte e as funções personalizadas mais complexas. Essas funções auxiliares podem ser muito úteis para certas conversões de tipo de dados.

Se você tem acompanhado, notará que não fiz nada com as colunas de ``date`` ou a coluna ``Jan Units``. Ambos podem ser convertidos simplesmente usando funções de pandas incorporadas, como ``pd.to_numeric()`` e ``pd.to_datetime()``.

O motivo pelo qual a conversão de ``Jan Unit`` é problemática é a inclusão de um valor não numérico na coluna. Se tentássemos usar astype(), obteríamos um erro (como descrito anteriormente). A função ``pd.to_numeric()`` pode lidar com esses valores com mais facilidade:

### 3.1 pd.to_numeric()

In [27]:
df['Jan Units']

0       500
1       700
2       125
3        75
4    Closed
Name: Jan Units, dtype: object

In [28]:
pd.to_numeric(df['Jan Units'], errors='coerce')

0    500.0
1    700.0
2    125.0
3     75.0
4      NaN
Name: Jan Units, dtype: float64

- Note que a o atributo Closed ficou NaN após a formatação



### 3.2 fillna()
- Filtra valores NaN

In [29]:
pd.to_numeric(df['Jan Units'], errors='coerce').fillna(0)

0    500.0
1    700.0
2    125.0
3     75.0
4      0.0
Name: Jan Units, dtype: float64

A conversão final que abordarei é converter as colunas separadas de mês, dia e ano em um datetime. A função pandas ``pd.to_datetime()`` é bastante configurável, mas também bastante inteligente por padrão.

### 3.3 pd_to_datetime()

In [30]:
pd.to_datetime(df[['Month', 'Day', 'Year']])

0   2015-01-10
1   2014-06-15
2   2016-03-29
3   2015-10-27
4   2014-02-02
dtype: datetime64[ns]

Nesse caso, a função combina as colunas em uma nova série do dtype ``datetime64`` apropriado.

Precisamos ter certeza de atribuir esses valores de volta ao dataframe:

In [31]:
df["Start_Date"] = pd.to_datetime(df[['Month', 'Day', 'Year']])
df["Jan Units"] = pd.to_numeric(df['Jan Units'], errors='coerce').fillna(0)

In [32]:
df.dtypes

Customer Number             int32
Customer Name              object
2016                      float64
2017                      float64
Percent Growth            float64
Jan Units                 float64
Month                       int64
Day                         int64
Year                        int64
Active                       bool
Start_Date         datetime64[ns]
dtype: object

In [33]:
df.head()

Customer Number     Customer Name      2016       2017  Percent Growth  \
0            10002  Quest Industries  125000.0   162500.0            0.30   
1           552278    Smith Plumbing  920000.0  1012000.0            0.10   
2            23477   ACME Industrial   50000.0    62500.0            0.25   
3            24900        Brekke LTD  350000.0   490000.0            0.04   
4           651029         Harbor Co   15000.0    12750.0           -0.15   

   Jan Units  Month  Day  Year  Active Start_Date  
0      500.0      1   10  2015    True 2015-01-10  
1      700.0      6   15  2014    True 2014-06-15  
2      125.0      3   29  2016    True 2016-03-29  
3       75.0     10   27  2015    True 2015-10-27  
4        0.0      2    2  2014   False 2014-02-02

# 4. Formatando Dados no Read_CSV


Os conceitos básicos do uso de ``astype()`` e funções customizadas podem ser incluídos muito cedo no processo de entrada de dados. Se você tem um arquivo de dados que pretende processar repetidamente e sempre vem no mesmo formato, você pode definir o ``dtype`` e os ``converters``a serem aplicados na leitura dos dados. É útil pensar em ``dtype`` como executando ``astype()`` nos dados. Os argumentos dos conversores permitem que você aplique funções às várias colunas de entrada semelhantes às abordagens descritas acima.

É importante observar que você só pode aplicar um dtype ou uma função de conversão a uma coluna especificada uma vez usando essa abordagem. Se você tentar aplicar ambos à mesma coluna, o dtype será ignorado.

Aqui está um exemplo simplificado que faz quase toda a conversão no momento em que os dados são lidos no dataframe:

In [34]:
df_2 = pd.read_csv(filename,
                   dtype={'Customer Number': 'int'},
                   converters={'2016': convert_currency,
                               '2017': convert_currency,
                               'Percent Growth': convert_percent,
                               'Jan Units': lambda x: pd.to_numeric(x, errors='coerce'),
                               'Active': lambda x: np.where(x == "Y", True, False)
                              },
                  parse_dates={'Start_Date':['Month', 'Day', 'Year']}
                  )

df_2.head()

Start_Date  Customer Number     Customer Name      2016       2017  \
0 2015-01-10            10002  Quest Industries  125000.0   162500.0   
1 2014-06-15           552278    Smith Plumbing  920000.0  1012000.0   
2 2016-03-29            23477   ACME Industrial   50000.0    62500.0   
3 2015-10-27            24900        Brekke LTD  350000.0   490000.0   
4 2014-02-02           651029         Harbor Co   15000.0    12750.0   

   Percent Growth  Jan Units Active  
0            0.30      500.0   True  
1            0.10      700.0   True  
2            0.25      125.0   True  
3            0.04       75.0   True  
4           -0.15        NaN  False